In [1]:
# 检查torch的安装以及gpu的使用
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# 检查MMAction2的安装
import mmaction
print(mmaction.__version__)

# 检查mmcv的安装
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.10.0+cu113 True
0.15.0
11.3
MSVC 192930137


In [2]:
from mmaction.apis import inference_recognizer, init_recognizer

# 选择tsn对应的配置文件
config = 'configs/recognition/swin/swin_base_patch244_window877_kinetics400_1k.py'
# 加载上面下载的checkpoint文件
checkpoint =  'checkpoints/swin_base_patch244_window877_kinetics400_1k.pth'
# 初始化模型
print(config)
model = init_recognizer(config, checkpoint, device='cuda:1')

configs/recognition/swin/swin_base_patch244_window877_kinetics400_1k.py


C:\Users\15281\anaconda3\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


load checkpoint from local path: checkpoints/swin_base_patch244_window877_kinetics400_1k.pth


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import random

# Function to read and randomly split a txt file
def read_and_split_txt(file_path, test_size=0.2):
    # Read the txt file into a pandas DataFrame
    data = pd.read_csv(file_path, delimiter=",", header=None)
    
    # Randomly shuffle the DataFrame
    data = data.sample(frac=1).reset_index(drop=True)
    
    # Split the data into features and target variable
    X = data  # Assuming the last column is the target variable
    y = data.iloc[:, -1]
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    
    return X_train, X_test, y_train, y_test

# Example usage
file_path = './video_data2/all.csv'
X_train, X_test, y_train, y_test = read_and_split_txt(file_path)
X_train.to_csv('./video_data2/cu/train.txt', index=False, header=False, sep=' ')
X_test.to_csv('./video_data2/cu/val_file.txt', index=False, header=False, sep=' ')

In [4]:
from mmcv.runner import set_random_seed
# 获得tsn对应的配置文件cfg
from mmcv import Config
cfg = Config.fromfile('./configs/recognition/swin/swin_base_patch244_window877_kinetics400_1k.py')
# 修改数据集类型和各个文件路径
cfg.dataset_type = 'VideoDataset'
cfg.data_root = './video_data2/train_video/'
cfg.data_root_val = './video_data2/val_video/'
cfg.ann_file_train = './video_data2/train.txt'
cfg.ann_file_val = './video_data2/val_file.txt'
cfg.ann_file_test = './video_data2/val_file.txt'


cfg.data.test.type = 'VideoDataset'
cfg.data.test.ann_file = './video_data2/val_file.txt'
cfg.data.test.data_prefix = './video_data2/val_video/'

cfg.data.train.type = 'VideoDataset'
cfg.data.train.ann_file = './video_data2/train.txt'
cfg.data.train.data_prefix = './video_data2/train_video/'
cfg.data.train.data_prefix = './video_data2/train_video/'

cfg.data.val.type = 'VideoDataset'
cfg.data.val.ann_file = './video_data2/val_file.txt'
cfg.data.val.data_prefix = './video_data2/val_video/'

# 这里用于确认是否使用到omnisource训练
cfg.setdefault('omnisource', False)
# 修改cls_head中类别数为3
cfg.model.cls_head.num_classes = 3
# 使用预训练好的tsn模型
cfg.load_from = 'checkpoints/swin_base_patch244_window877_kinetics400_1k.pth'
# 设置工作目录
cfg.work_dir = './tutorial_exps'

# 由于是单卡训练，修改对应的lr
#cfg.data.videos_per_gpu = cfg.data.videos_per_gpu // 16
cfg.data.videos_per_gpu = 2
cfg.optimizer.lr = cfg.optimizer.lr / 8 / 16
#设置epoches
cfg.total_epochs = 50
# 设置存档点间隔减少存储空间的消耗
cfg.checkpoint_config.interval = 10
# 设置日志打印间隔减少打印时间
cfg.log_config.interval = 5

# 固定随机种子使得结果可复现
cfg.seed = 40
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1,2)

# 打印所有的配置参数
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='Recognizer3D',
    backbone=dict(
        type='SwinTransformer3D',
        patch_size=(2, 4, 4),
        embed_dim=128,
        depths=[2, 2, 18, 2],
        num_heads=[4, 8, 16, 32],
        window_size=(8, 7, 7),
        mlp_ratio=4.0,
        qkv_bias=True,
        qk_scale=None,
        drop_rate=0.0,
        attn_drop_rate=0.0,
        drop_path_rate=0.3,
        patch_norm=True),
    cls_head=dict(
        type='I3DHead',
        in_channels=1024,
        num_classes=3,
        spatial_type='avg',
        dropout_ratio=0.5),
    test_cfg=dict(average_clips='prob', max_testing_views=4))
checkpoint_config = dict(interval=10)
log_config = dict(interval=5, hooks=[dict(type='TextLoggerHook')])
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = 'checkpoints/swin_base_patch244_window877_kinetics400_1k.pth'
resume_from = None
workflow = [('train', 1)]
dataset_type = 'VideoDataset'
data_root = './video_data2/train_video/'
data_root_val = './

In [ ]:
import os.path as osp

from mmaction.datasets import build_dataset
from mmaction.models import build_model
from mmaction.apis import train_model

import mmcv
import os

# Get a list of all files in the current directory
files_in_directory = os.listdir()
filtered_files = [file for file in files_in_directory if file.endswith(".pt")]

# Loop through the list and remove each file
for file in filtered_files:
    os.remove(file)
    #print(f"Removed {file}")
# 构建数据集
datasets = [build_dataset(cfg.data.train)]
# 构建动作识别模型
model = build_model(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))

# 创建工作目录并训练模型
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_model(model, datasets, cfg, distributed=False, validate= True)